"Playground" for implementing Monte Carlo Dropout in Pytorch, as not possible in native pytorch.

Some links:
- https://stackoverflow.com/questions/63285197/measuring-uncertainty-using-mc-dropout-on-pytorch
- https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/
- https://discuss.pytorch.org/t/measuring-uncertainty-using-mc-dropout/91753

In [1]:
#imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import h5py
import numpy as np

In [2]:
#dataloading and preprocessing
with h5py.File('data/tactmat.h5', 'r') as dataset:
    samples = dataset['samples'][:]
    materials = dataset['materials'][:]
    materials = [m.decode('utf-8') for m in materials]
train_samples = samples[:, :80,...]  # (36, 80, 1000, 4, 4)
test_samples = samples[:, 80:,...]  # (36, 20, 1000, 4, 4)

# Flatten data
train_samples = train_samples.reshape((36*80, 1000, 16))  # (2880, 1000, 16)
test_samples = test_samples.reshape((36*20, 1000, 16))  # (720, 1000, 16)

# Create labels
train_labels = np.repeat(np.arange(36), 80)  # (2880,)
test_labels = np.repeat(np.arange(36), 20)  # (720,)

# One-hot encoding not needed for pytorch CrossEntropyLoss
# train_labels = np.eye(36)[train_labels]  # (2880, 36)
# test_labels = np.eye(36)[test_labels]  # (720, 36)
# adapt to pytorch tensor shape
train_samples = torch.tensor(train_samples.reshape(2880, 1, 1000, 16), dtype=torch.float32)
test_samples = torch.tensor(test_samples.reshape(720, 1, 1000, 16), dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'data/tactmat.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#model definition
class MCTestTacNet(nn.Module):
    def __init__(self):
        super(MCTestTacNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(15, 5))
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=(10, 1), stride=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(15, 5))
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=(10, 1), stride=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(15, 5))
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=(10, 1), stride=1)
        self.dropout = nn.Dropout(0.8)
        self.fc1 = nn.Linear(128 * 922 * 12, 512)  # Adjust flattened size if needed
        self.bn4 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 36)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.bn4(self.fc1(self.dropout(x))))
        x = F.softmax(self.fc2(x), dim=1)
        return x

model = MCTestTacNet()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check if CUDA is available and move model to GPU if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
